In [1]:
import gradio as gr
import importlib
import os
from IPython.display import Markdown
from python_scripts import llm_rag, machine_translation, text_to_speech, whisper_setup, get_audio, utils
import numpy as np
from scipy.io.wavfile import write
# r8_eYbLLc9wPQ1qa4dUK8eXyewFKHvGAck1h9LZl

# Define paths dynamically
PATH = os.getcwd().replace('\\\\', '/')
root_path = PATH + '\\Datasets\\MeDAL'
data_path = root_path + '\\training_data(2).jsonl'
print(data_path)
audio_path = PATH + '\\Datasets\\Audio_Files'

c:\Users\Admin\OneDrive\Documents\GitHub\CS-5302-Project-Group-15\Datasets\MeDAL\training_data(2).jsonl


In [2]:

importlib.reload(llm_rag)
# importlib.reload(machine_translation)
# importlib.reload(text_to_speech)
# importlib.reload(whisper_setup)
# importlib.reload(get_audio)
# importlib.reload(utils)


<module 'python_scripts.llm_rag' from 'c:\\Users\\Admin\\OneDrive\\Documents\\GitHub\\CS-5302-Project-Group-15\\python_scripts\\llm_rag.py'>

In [3]:
def SMTS(audio):
    try:

        file_path = 'output.wav'
        write(file_path, data = np.array(audio[1], dtype = np.int16), rate = audio[0])
        audio_processed = utils.preprocess_audio(file_path)

        # Step 2: Transcribe Query to English
        whisper_models = ["tiny", "base", "small", "medium", "large"]
        
        transcript = whisper_setup.transcribe_audio(audio_processed, ['tiny'])

        # Step 3: Feed query into the LLM
        models = { 
        'llama_ours': 'ubaidtariq8/llama2-med-genai', # fine tuned model
        'llama_13b': 'a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5',
        'mixtral': 'mistralai/mixtral-8x7b-instruct-v0.1',
        'llama_70b': 'meta/llama-2-70b-chat:2796ee9483c3fd7aa2e171d38f4ca12251a30609463dcfd4cd76703f22e96cdf'
        }

        print("0")
        model = llm_rag.DocumentEmbeddingPipeline(model_version = models['mixtral'], chroma_path = root_path)
        print("1")
        model.setup_environment()
        print("2")
        model.prepare_documents(data_path = data_path, collection_name = "muqeen", joining = True, persistent = False)
        print("3")
        model.embed_and_index()

        print("4")
        response = model.query_data(query = transcript['tiny'])
        print("5")

        # # Step 4: Translate it back to the user language
        translated_text = machine_translation.translate_text(text = " ".join(response), src_lang = 'en', trg_lang = transcript['lang'])

        # Step 5: Now speak the response in the user's language
        audio_answer_path = audio_path + '/audio.wav'
        text_to_speech.multilingual_text_to_speech(text = translated_text, filepath = audio_answer_path)
        utils.sasti_harkat(audio_answer_path)
        # display(Markdown(f"<b>{translated_text}</b>"))
        return transcript['tiny'], utils.play_wav(audio_answer_path)
    except Exception as e:
        print("An error occurred:", e)

In [4]:
demo = gr.Interface(
    SMTS,
    gr.Audio(sources = ["microphone"]),
    ["text", "audio"],
)

demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\Admin\anaconda3\envs\genAI\Lib\site-packages\uvicorn\protocols\http\httptools_impl.py", line 411, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Admin\anaconda3\envs\genAI\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 69, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Admin\anaconda3\envs\genAI\Lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\Admin\anaconda3\envs\genAI\Lib\site-packages\starlette\applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\Admin\anaconda3\envs\genAI\Lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  Fi

0
1
2
talha muqeem
[]
collection nahi hai
c:\Users\Admin\OneDrive\Documents\GitHub\CS-5302-Project-Group-15\Datasets\MeDAL\diagnosis_data_combined.jsonl
c:\Users\Admin\OneDrive\Documents\GitHub\CS-5302-Project-Group-15\Datasets\MeDAL\training_data(2).jsonl
RR
An error occurred: No files found in c:\Users\Admin\OneDrive\Documents\GitHub\CS-5302-Project-Group-15\Datasets\MeDAL.


Traceback (most recent call last):
  File "c:\Users\Admin\anaconda3\envs\genAI\Lib\site-packages\gradio\queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Admin\anaconda3\envs\genAI\Lib\site-packages\gradio\route_utils.py", line 261, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Admin\anaconda3\envs\genAI\Lib\site-packages\gradio\blocks.py", line 1797, in process_api
    data = await self.postprocess_data(fn_index, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Admin\anaconda3\envs\genAI\Lib\site-packages\gradio\blocks.py", line 1565, in postprocess_data
    self.validate_outputs(fn_index, predictions)  # type: ignore
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Admin\anaconda3\envs\genAI\Lib\s

In [ ]:
C:\Users\Admin\OneDrive\Documents\GitHub\CS-5302-Project-Group-15\Datasets\MeDAL